In [12]:
import os
import pandas as pd
import requests
import json
from pprint import pprint

In [4]:
# json downlad folder
jsonFolder = os.path.join("bird_data","json")
# audio folder
audioFolder = os.path.join("bird_data","audio")

In [5]:
birds_common_name = ["American Goldfinch", "Barn Swallow", "Blue Jay", "Blue-grey Gnatcatcher", 
                     "Broad-winged Hawk", "Carolina Chickadee", "Eastern Bluebird", "Northern Cardinal", 
                     "Rose-breasted Grosbeak","Ruby-crowned Kinglet", "Tufted Titmouse"]

birds_sci_name = ["Spinus tristis", "Hirundo rustica","Cyanocitta cristata", "Polioptila caerulea", 
                  "Buteo platypterus", "Poecile carolinensis", "Sialia sialis", "Cardinalis cardinalis", 
                  "Pheucticus ludovicianus", "Regulus calendula","Baeolophus bicolor" ]

queryBirds = dict(zip(birds_common_name, birds_sci_name))

In [6]:
# function to build the query urls
def queryURL(queryBirds):
    endpoint = "https://www.xeno-canto.org/api/2/recordings?query="
    birdy = []
    for v in queryBirds.values():
        aux=v.split()
        birdy.append(aux)
    queryURL = []
    
    for i,j in birdy:
        url = f"{endpoint}+{i}+{j}+type:call+q_gt:C"
        queryURL.append(url)
        
    return queryURL
    
    

In [7]:
# function to request data by bird species
def requestBird(url):
    payload = {}
    response = requests.request("GET", url, data = payload).json()
    return response

In [8]:
# funcion to save json from the request
def jsonBird(response,bird, path):
    file_name = f'{bird}.json'
    file_path = os.path.join(path, file_name)
    with open(file_path,'w', encoding='utf-8') as f:
        json.dump(response,f, ensure_ascii=False, indent=4)
    f.close()
    

In [9]:
# request data
def xenoRequest(queryBirds, folder):
    bird_name = [k for k in queryBirds.keys()]
    birds = []
    for name in bird_name:
        bird_name = name.replace(" ","_")
        birds.append(bird_name)
    qURL = queryURL(queryBirds)
    for q,b in zip(qURL,birds):
        response = requestBird(q)
        jsonBird(response, b, folder)
    

In [10]:
xenoRequest(queryBirds, jsonFolder)

url = "http://www.xeno-canto.org/555073/download"
audio = requests.get(url, allow_redirects=True)
open("test.mp3","wb").write(audio.content)